# VGG16

In [ ]:
import torch
import torch.nn as nn

cfg = {
    'A' : [64,     'M', 128,      'M', 256, 256,           'M', 512, 512,           'M', 512, 512,           'M'],
    'B' : [64, 64, 'M', 128, 128, 'M', 256, 256,           'M', 512, 512,           'M', 512, 512,           'M'],
    'D' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256,      'M', 512, 512, 512,      'M', 512, 512, 512,      'M'],
    'E' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}

class VGG(nn.Module):

    def __init__(self, features, num_class=100):
        super().__init__()
        self.features = features

        self.classifier = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_class)
        )

    def forward(self, x):
        output = self.features(x)
        output = output.view(output.size()[0], -1)
        output = self.classifier(output)

        return output

def make_layers(cfg, batch_norm=False):
    layers = []

    input_channel = 3
    for l in cfg:
        if l == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            continue

        layers += [nn.Conv2d(input_channel, l, kernel_size=3, padding=1)]

        if batch_norm:
            layers += [nn.BatchNorm2d(l)]

        layers += [nn.ReLU(inplace=True)]
        input_channel = l

    return nn.Sequential(*layers)

def vgg16_bn():
    return VGG(make_layers(cfg['D'], batch_norm=True))

# Training

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset

# Define the data transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Download the CIFAR-100 dataset
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

# Split the dataset into training, validation, and test sets
train_size = int(0.8 * len(train_dataset))
val_size = int(0.1 * len(train_dataset))
test_size = len(train_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(train_dataset, [train_size, val_size, test_size])

# Create balanced DataLoader for training, validation, and test sets
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
! git clone https://github.com/K-Hooshanfar/pytorch-cifar100

Cloning into 'pytorch-cifar100'...
remote: Enumerating objects: 1043, done.
remote: Counting objects: 100% (1043/1043), done.
remote: Compressing objects: 100% (386/386), done.
remote: Total 1043 (delta 658), reused 1008 (delta 645), pack-reused 0
Receiving objects: 100% (1043/1043), 492.58 KiB | 1.94 MiB/s, done.
Resolving deltas: 100% (658/658), done.


In [ ]:
%cd pytorch-cifar100


/content/pytorch-cifar100


In [ ]:
!python train.py -net vgg16 -gpu

2023-12-01 12:46:15.279934: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 12:46:15.279987: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 12:46:15.280024: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 12:46:16.347360: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Files already downloaded and verified
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system i

## Save weights

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import shutil

source_path = '/content/pytorch-cifar100/checkpoint/vgg16/Friday_01_December_2023_12h_46m_17s/vgg16-200-regular.pth'
destination_path = '/content/gdrive/My Drive/vgg16-200-regular.pth'

shutil.copyfile(source_path, destination_path)


'/content/gdrive/My Drive/vgg16-200-regular.pth'

In [ ]:
model = vgg16_bn()

# Load pre-trained weights
pretrained_weights_path = '/content/pytorch-cifar100/checkpoint/vgg16/Friday_01_December_2023_12h_46m_17s/vgg16-200-regular.pth'
model.load_state_dict(torch.load(pretrained_weights_path))

<All keys matched successfully>

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

# Define data transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-100 dataset
cifar100_test = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)


# Create PyTorch data loaders
batch_size = 128
val_loader = torch.utils.data.DataLoader(cifar100_test, batch_size=batch_size, shuffle=False)

# Check the number of samples in each set
print(f"Test set size: {len(cifar100_test)}")

100%|██████████| 169001437/169001437 [00:13<00:00, 12346764.03it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Test set size: 10000


In [1]:
model.eval()
correct_test = 0
total_test = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Final Test Accuracy: {test_accuracy:.2f}%")


Final Test Accuracy: 72.29%


# Load Model and Get Feature Latent

In [18]:
from torch.utils.data.sampler import SubsetRandomSampler
import torch
from torchvision import datasets, transforms
import random
import numpy as np

# Set random seeds for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

# Load the CIFAR-100 dataset and create a balanced subset
transform = transforms.Compose([transforms.ToTensor()])
# Load CIFAR-100 dataset
cifar100_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(cifar100_dataset))
val_size = len(cifar100_dataset) - train_size
cifar100_traindataset, cifar100_valdataset = torch.utils.data.random_split(cifar100_dataset, [train_size, val_size])

# Define the subset size
subset_fraction = 0.9
subset_size_train = int(subset_fraction * len(cifar100_traindataset))

class_indices = list(range(len(cifar100_traindataset.dataset.classes)))
class_subset_size = int(subset_size_train / len(cifar100_traindataset.dataset.classes))

class_sampler_indices_train = []

for class_index in class_indices:
    class_indices_list_train = [i for i, label in enumerate(cifar100_traindataset.dataset.targets) if label == class_index]
    class_sampler_indices_train.extend(class_indices_list_train[:class_subset_size])

train_sampler = SubsetRandomSampler(class_sampler_indices_train)

batch_size = 256
train_loader = torch.utils.data.DataLoader(cifar100_traindataset, batch_size=batch_size, sampler=train_sampler)

# Check the number of samples in the balanced train set
print(f"Balanced Train set size: {len(train_loader.sampler)}")

Files already downloaded and verified
Balanced Train set size: 36000


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the VGG16 model
model = vgg16_bn()
model.to(device)

# Load pre-trained weights
pretrained_weights_path = '/content/pytorch-cifar100/checkpoint/vgg16/Friday_01_December_2023_12h_46m_17s/vgg16-200-regular.pth'
model.load_state_dict(torch.load(pretrained_weights_path))
model = nn.Sequential(*list(model.children())[:-1])

# Define your CIFAR-100 dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

features = []
labels = []

# Set the model to evaluation mode
model.eval()

with torch.no_grad():
    for inputs, targets in tqdm(train_loader):
        if torch.cuda.is_available():
            inputs = inputs.to('cuda')

        # Forward pass through the model.features
        features_batch = model(inputs)

        # Append the extracted features and labels
        features.append(features_batch)
        labels.append(targets)

        # Release GPU memory
        del inputs
        torch.cuda.empty_cache()


100%|██████████| 79/79 [00:07<00:00, 11.01it/s]


In [ ]:
# Stack and reshape the extracted features
features = torch.cat(features)
features = features.view(features.size(0), -1)
labels = torch.cat(labels)
labels = labels.unsqueeze(1)

# Metrics for train set

In [3]:
!git clone https://github.com/Arhosseini77/data_complexity_measures

Cloning into 'data_complexity_measures'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 140 (delta 80), reused 33 (delta 9), pack-reused 0
Receiving objects: 100% (140/140), 144.00 KiB | 4.23 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from torch.utils.data import Subset, DataLoader
import numpy as np
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data.sampler import SubsetRandomSampler

from data_complexity_measures.models.ARH_SeparationIndex import ARH_SeparationIndex

In [ ]:
# Create Instance of class
si_calculator = ARH_SeparationIndex(features, labels, normalize=True)

Data has been normalized


# SI

In [ ]:
si_data = si_calculator.si()
print(si_data)

Calculating SI: 100%|██████████| 40000/40000 [00:02<00:00, 17628.01it/s]

0.338725


# Calc High order SI (order = 2)

In [ ]:
si_high_order_2_data = si_calculator.high_order_si(order=2)
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 20000/20000 [00:02<00:00, 7028.67it/s] 

0.21879999339580536


# High order soft SI (order=2)

In [ ]:
si_soft_order_2_data = si_calculator.soft_order_si(order=2)
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 20000/20000 [00:01<00:00, 10047.12it/s]

Soft Order(2) SI : 0.3128249943256378


# Center Based SI

In [ ]:
center_si_data = si_calculator.center_si()
print("Center SI:", center_si_data)

Calculating Class Centers: 100%|██████████| 100/100 [00:00<00:00, 6109.25it/s]

Center SI: 0.3668999969959259


# Anti SI (order = 2)

In [ ]:
anti_si = si_calculator.anti_si(order=2)
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 20000/20000 [00:01<00:00, 11169.87it/s]

anti_si: 0.5931499600410461


# Metrics for test set

In [5]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
# Define data transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-100 dataset
cifar100_test = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

# Create PyTorch data loaders
batch_size = 128
test_loader = torch.utils.data.DataLoader(cifar100_test, batch_size=batch_size, shuffle=False)

100%|██████████| 169001437/169001437 [00:02<00:00, 66013335.15it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the VGG16 model
model = vgg16_bn()
model.to(device)

# Load pre-trained weights
pretrained_weights_path = '/content/pytorch-cifar100/checkpoint/vgg16/Friday_01_December_2023_12h_46m_17s/vgg16-200-regular.pth'
model.load_state_dict(torch.load(pretrained_weights_path))
model = nn.Sequential(*list(model.children())[:-1])

# Define your CIFAR-100 dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

features = []
labels = []

# Set the model to evaluation mode
model.eval()

with torch.no_grad():
    for inputs, targets in tqdm(test_loader):
        if torch.cuda.is_available():
            inputs = inputs.to('cuda')

        # Forward pass through the model.features
        features_batch = model(inputs)

        # Append the extracted features and labels
        features.append(features_batch)
        labels.append(targets)

        # Release GPU memory
        del inputs
        torch.cuda.empty_cache()


In [ ]:
# Stack and reshape the extracted features
features = torch.cat(features)
features = features.view(features.size(0), -1)
labels = torch.cat(labels)
labels = labels.unsqueeze(1)

In [ ]:
# Create Instance of class
si_calculator = ARH_SeparationIndex(features, labels, normalize=True)

Data has been normalized


# SI

In [ ]:
si_data = si_calculator.si()
print(si_data)

Calculating SI: 100%|██████████| 10000/10000 [00:00<00:00, 17869.21it/s]

0.5724


# Calc High order SI (order = 2)

In [ ]:
si_high_order_2_data = si_calculator.high_order_si(order=2)
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 10000/10000 [00:00<00:00, 13112.51it/s]

0.48099997639656067


# High order soft SI (order=2)

In [ ]:
si_soft_order_2_data = si_calculator.soft_order_si(order=2)
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 10000/10000 [00:01<00:00, 8485.99it/s]

Soft Order(2) SI : 0.5684499740600586


# Center Based SI

In [ ]:
center_si_data = si_calculator.center_si()
print("Center SI:", center_si_data)

Calculating Class Centers: 100%|██████████| 100/100 [00:00<00:00, 3567.62it/s]

Center SI: 0.6365000009536743


# Anti SI (order = 2)

In [ ]:
anti_si = si_calculator.anti_si(order=2)
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 10000/10000 [00:00<00:00, 12083.23it/s]

anti_si: 0.3440999984741211


# Pretrained VGG16

In [3]:
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100

# Load pre-trained models
vgg16_model = models.vgg16(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Remove fully connected layers
vgg16_features = torch.nn.Sequential(*(list(vgg16_model.features.children())))

In [19]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def extract_features(model, dataloader):
    model.eval()
    model.to(device)
    features = []
    labels = []

    with torch.no_grad():
        for images, targets in tqdm(dataloader):
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            features.append(outputs.squeeze())
            labels.append(targets)

    features = torch.cat(features)
    labels = torch.cat(labels)

    return features, labels

# Move models to GPU
vgg16_features.to(device)

resnet18_features, resnet18_labels = extract_features(vgg16_features, train_loader)

100%|██████████| 141/141 [00:05<00:00, 26.12it/s]


In [6]:
si_calculator = ARH_SeparationIndex(resnet18_features, resnet18_labels, normalize=True)

Data has been normalized


# SI

In [19]:
si_data = si_calculator.si_batch(batch_size=2000)
print(si_data)

Calculating SI: 100%|██████████| 18/18 [00:00<00:00, 1133.03it/s]

0.23000000417232513


# Calc High order SI (order = 2)

In [20]:
si_high_order_2_data = si_calculator.high_order_si_batch(order=2, batch_size=2000)
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 18/18 [00:00<00:00, 1102.57it/s]


0.10044444352388382


# High order soft SI (order=2)

In [21]:
si_soft_order_2_data = si_calculator.soft_order_si_batch(order=2 , batch_size=2000)
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 18/18 [00:00<00:00, 1179.91it/s]


Soft Order(2) SI : 0.21113888919353485


# Center Based SI

In [22]:
center_si_data = si_calculator.center_si_batch(batch_size=2000)
print("Center SI:", center_si_data)

Calculating CSI: 100%|██████████| 18/18 [00:00<00:00, 2658.08it/s]

Center SI: 0.277444452047348


# Anti SI (order = 2)

In [7]:
anti_si = si_calculator.anti_si(order=2)
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 8000/8000 [00:00<00:00, 14707.54it/s]

anti_si: 0.7393750548362732


Test

In [6]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def extract_features(model, dataloader):
    model.eval()
    model.to(device)
    features = []
    labels = []

    with torch.no_grad():
        for images, targets in tqdm(dataloader):
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            features.append(outputs.squeeze())
            labels.append(targets)

    features = torch.cat(features)
    labels = torch.cat(labels)

    return features, labels

# Move models to GPU
vgg16_features.to(device)

resnet18_features, resnet18_labels = extract_features(vgg16_features, test_loader)

100%|██████████| 79/79 [00:09<00:00,  8.11it/s]


In [7]:
si_calculator = ARH_SeparationIndex(resnet18_features, resnet18_labels, normalize=True)

Data has been normalized


In [8]:
si_data = si_calculator.si_batch(batch_size = 2000)
print(":")
print(si_data)

Calculating SI: 100%|██████████| 5/5 [00:00<00:00, 450.31it/s]

:
0.22019998729228973


In [9]:
si_high_order_2_data = si_calculator.high_order_si_batch(order=2 , batch_size = 2000)
print(":")
print(si_high_order_2_data)

Computing High Order SI: 100%|██████████| 5/5 [00:01<00:00,  4.40it/s]

:
0.09479999542236328


In [11]:
si_soft_order_2_data = si_calculator.soft_order_si_batch(order=2 ,batch_size = 2000 )
print(":")
print("Soft Order(2) SI :", si_soft_order_2_data)

Calculating Soft Order SI: 100%|██████████| 5/5 [00:00<00:00, 561.05it/s]

:
Soft Order(2) SI : 0.20469999313354492


In [12]:
center_si_data = si_calculator.center_si_batch(batch_size = 200)
print(":")
print("Center SI:", center_si_data)

Calculating CSI: 100%|██████████| 50/50 [00:00<00:00, 3091.09it/s]

:
Center SI: 0.3497999906539917


In [13]:
anti_si = si_calculator.anti_si(order=2)
print("anti_si:", anti_si)

Calculating Anti-SI: 100%|██████████| 10000/10000 [00:00<00:00, 13853.41it/s]

anti_si: 0.6855999827384949
